In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import requests
import dask.dataframe as dd
from transformers import pipeline

In [25]:
pipe = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [47]:
def apply_pipe(caption):
    return pipe("Provide the mood for the description of a piece of music: " + caption)

In [48]:
df = dd.read_csv('DATASET/Captioned/*.csv')
df['mood'] = df['captioning'].apply(apply_pipe, meta=('mood', 'int'))
df = df.compute()

In [56]:
np.array(df.sample(2))

array([['pop.00080.wav',
        '[0:00-10:00]\nThis is an R&B/pop music piece. There is a female vocalist singing melodically. The melody is being played by the keyboard while the bass guitar is playing in the background. The rhythm is provided by a simple electronic drum beat. The atmosphere is urban. This piece could be used in the soundtrack of a teenage drama movie that takes place in the big city. \n \n[10:00-20:00]\nThis is an R&B/pop music piece. There is a female vocalist singing melodically in the lead. The melody is being played by the keyboard while the bass guitar is playing in the background. The rhythm is provided by a simple electronic drum beat. The atmosphere is sentimental. This piece could be used in the soundtrack of a romantic comedy movie, especially during the scenes where a character is going through the feeling of heartbreak. \n \n[20:00-30:00]\nThis is an R&B/pop music piece. There is a female vocalist singing melodically in the lead. The melody is being play

## Packaging

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import requests
import dask.dataframe as dd

In [2]:
def get_mood(clips_descriptions):
    # API keys and tokens
    API_URL_Z = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
    headers_Z = {"Authorization": "Bearer "}

    # Prompt tuning
    Z_Query_Final = f"""<|system|>
        Given the descriptions about each ten-second clips of a song, tell me what the overall mood of the song is</s>
        <|user|>
        {clips_descriptions}</s>
        <|assistant|>"""
    
    # Generate response
    Zephyr_B_Beta_Final_Generated_Response = requests.post(API_URL_Z, headers=headers_Z, json={"inputs": Z_Query_Final}).json()
    
    return Zephyr_B_Beta_Final_Generated_Response[0]["generated_text"][len(Z_Query_Final):]

In [7]:
df = dd.read_csv('DATASET/Captioned/*.csv') # this is just to get a song description
print(get_mood(list(df["captioning"])[0]))


Based on the descriptions provided, the overall mood of the song is groovy and could be playing in the background at a rock bar. All three clips feature a male vocalist singing melodically, with the electric guitar and bass guitar playing a prominent role in the music. The rhythmic background consists of a slow tempo beat, either from an acoustic drum or a blues acoustic drum. The use of the word "groovy" in each description suggests a relaxed and enjoyable atmosphere, making it suitable for a rock bar setting. Therefore, it can be concluded that the overall mood of the song is groovy and perfect for creating a laid-back ambiance in a rock bar.
